In [19]:
import struct
import zlib

import pandas as pd
import numpy as np
import clipboard, re
import olefile

In [149]:
columns = ['구분', '진동속도(cm/s)', '진동레벨[dB(V)]', '소음[dB(A)]','허용기준', '비고']
columns2 = ['최저치', '최고치']

clipboard_list = [i for i in list(filter(lambda v: v, clipboard.paste().replace("\r\n", " ").split(" "))) if i not in columns and i not in columns2]
clipboard_list = list(map(lambda x: np.nan if x == '-' else x, clipboard_list))

convert_list = []
for i in clipboard_list:
    try:
        convert_list.append(float(i))
    except:
        convert_list.append(i)

 # or i not in columns2
print(convert_list)
print(len(convert_list))

['11월3일', '다윗관부지경계', 0.02, 0.038, 42.8, 57.22, 66.41, 74.58, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.022, 0.1, 52.73, 67.25, 61.26, 71.47, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.045, 0.117, 52.14, 68.11, 69.36, 74.69, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월4일', '다윗관부지경계', 0.021, 0.028, 45.32, 53.89, 68.23, 73.13, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.024, 0.103, 51.0, 69.53, 67.85, 74.68, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.047, 0.145, 58.03, 68.71, 71.03, 74.81, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월7일', '다윗관부지경계', 0.021, 0.032, 45.17, 50.48, 71.0, 74.41, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.068, 0.142, 56.44, 66.99, 71.79, 74.99, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.031, 0.146, 41.79, 71.57, 50.48, 74.57, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월8일', '다윗관부지경계', nan, 0.023, nan, 49.52, nan, 69.46, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.048, 0.093, 52.75, 62.44, 65.51, 74.14, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.056, 0.152, 60.33, 69.93, 69.38, 74.7, '0.2cm/s75dB

In [150]:
# ClassficationByDate
classfication_dict = {}
data_list = []
current_data = []
date_key, location_key = None, None

for x, i in enumerate(convert_list):
    if re.match(r'\d+월\d+일', str(i)):
        date_key = i
    elif date_key and re.match(r'[\uAC00-\uD7A3]+', str(i)):
        location_key = i
    elif date_key and location_key:
        if len(data_list) >= 4:
            for idx, sublist in enumerate(data_list):
                if location_key not in classfication_dict:
                    classfication_dict[location_key] = {}
                if f'{date_key} {x}' not in classfication_dict[location_key]:
                    classfication_dict[location_key][f'{date_key} {x}'] = {}
                if idx == 0:
                    classfication_dict[location_key][f'{date_key} {x}']['일시'] = date_key                     
                for sub_idx, data in enumerate(sublist):
                    if idx == 3:
                        classfication_dict[location_key][f'{date_key} {x}'][f'{columns[idx + 1]}'] = data
                        classfication_dict[location_key][f'{date_key} {x}'][f'{columns[idx + 2]}'] = sublist[sub_idx + 1]
                        break
                    else:
                        classfication_dict[location_key][f'{date_key} {x}'][f'{columns[idx + 1]}{columns2[sub_idx]}'] = data

            data_list = []
        
        if not current_data or len(current_data) < 2:
            current_data.append(i)
        else:
            data_list.append(current_data)
            current_data = [i]

# for i in clipboard_list:
#     if re.match(r'\d+월\d+일', i):
#         if current_list:
#             classfication_list.append(current_list)
#             current_list = []
#             
#         current_list.append(i)
#     else:
#         current_list.append(i)
                

In [151]:
for i in classfication_dict:
    print(i, end=" ")

주택 식당부지경계 다윗관부지경계 

In [152]:
classfication_dict

{'주택': {'11월3일 12': {'일시': '11월3일',
   '진동속도(cm/s)최저치': 0.02,
   '진동속도(cm/s)최고치': 0.038,
   '진동레벨[dB(V)]최저치': 42.8,
   '진동레벨[dB(V)]최고치': 57.22,
   '소음[dB(A)]최저치': 66.41,
   '소음[dB(A)]최고치': 74.58,
   '허용기준': '0.2cm/s75dB(V)75dB(A)',
   '비고': 'O.K'},
  '11월4일 40': {'일시': '11월4일',
   '진동속도(cm/s)최저치': 0.021,
   '진동속도(cm/s)최고치': 0.028,
   '진동레벨[dB(V)]최저치': 45.32,
   '진동레벨[dB(V)]최고치': 53.89,
   '소음[dB(A)]최저치': 68.23,
   '소음[dB(A)]최고치': 73.13,
   '허용기준': '0.2cm/s75dB(V)75dB(A)',
   '비고': 'O.K'},
  '11월7일 68': {'일시': '11월7일',
   '진동속도(cm/s)최저치': 0.021,
   '진동속도(cm/s)최고치': 0.032,
   '진동레벨[dB(V)]최저치': 45.17,
   '진동레벨[dB(V)]최고치': 50.48,
   '소음[dB(A)]최저치': 71.0,
   '소음[dB(A)]최고치': 74.41,
   '허용기준': '0.2cm/s75dB(V)75dB(A)',
   '비고': 'O.K'},
  '11월8일 96': {'일시': '11월8일',
   '진동속도(cm/s)최저치': nan,
   '진동속도(cm/s)최고치': 0.023,
   '진동레벨[dB(V)]최저치': nan,
   '진동레벨[dB(V)]최고치': 49.52,
   '소음[dB(A)]최저치': nan,
   '소음[dB(A)]최고치': 69.46,
   '허용기준': '0.2cm/s75dB(V)75dB(A)',
   '비고': 'O.K'},
  '11월9일 124': {'일시': '

In [138]:
a = pd.DataFrame(classfication_dict['주택']).transpose()

In [139]:
a

,일시,진동속도(cm/s)최저치,진동속도(cm/s)최고치,진동레벨[dB(V)]최저치,진동레벨[dB(V)]최고치,소음[dB(A)]최저치,소음[dB(A)]최고치,허용기준,비고
12,11월3일,0.02,0.038,42.8,57.22,66.41,74.58,0.2cm/s75dB(V)75dB(A),O.K
40,11월4일,0.021,0.028,45.32,53.89,68.23,73.13,0.2cm/s75dB(V)75dB(A),O.K
68,11월7일,0.021,0.032,45.17,50.48,71.0,74.41,0.2cm/s75dB(V)75dB(A),O.K
96,11월8일,NaN,0.023,NaN,49.52,NaN,69.46,0.2cm/s75dB(V)75dB(A),O.K
124,11월9일,NaN,0.021,NaN,44.63,NaN,73.1,0.2cm/s75dB(V)75dB(A),O.K
152,11월14일,0.021,0.066,42.82,51.98,66.77,73.83,0.2cm/s75dB(V)75dB(A),O.K
180,11월15일,0.02,0.036,45.43,56.85,65.87,74.8,0.2cm/s75dB(V)75dB(A),O.K
208,11월17일,0.01,0.03,45.4,55.26,73.13,73.5,0.2cm/s75dB(V)75dB(A),O.K
236,11월20일,0.022,0.03,42.84,54.99,70.51,74.61,0.2cm/s75dB(V)75dB(A),O.K
264,11월23일,0.021,0.037,44.7,58.0,66.02,71.84,0.2cm/s75dB(V)75dB(A),O.K


In [ ]:
['11월3일', '다윗관부지경계', 0.02, 0.038, 42.8, 57.22, 66.41, 74.58, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.022, 0.1, 52.73, 67.25, 61.26, 71.47, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.045, 0.117, 52.14, 68.11, 69.36, 74.69, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월4일', '다윗관부지경계', 0.021, 0.028, 45.32, 53.89, 68.23, 73.13, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.024, 0.103, 51.0, 69.53, 67.85, 74.68, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.047, 0.145, 58.03, 68.71, 71.03, 74.81, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월7일', '다윗관부지경계', 0.021, 0.032, 45.17, 50.48, 71.0, 74.41, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.068, 0.142, 56.44, 66.99, 71.79, 74.99, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.031, 0.146, 41.79, 71.57, 50.48, 74.57, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월8일', '다윗관부지경계', nan, 0.023, nan, 49.52, nan, 69.46, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.048, 0.093, 52.75, 62.44, 65.51, 74.14, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.056, 0.152, 60.33, 69.93, 69.38, 74.7, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월9일', '다윗관부지경계', nan, 0.021, nan, 44.63, nan, 73.1, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.042, 0.129, 55.02, 68.94, 65.12, 74.71, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.05, 0.137, 59.93, 70.09, 70.96, 74.92, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월14일', '다윗관부지경계', 0.021, 0.066, 42.82, 51.98, 66.77, 73.83, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.02, 0.039, 42.15, 57.58, 66.09, 73.61, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.029, 0.127, 47.57, 66.51, 69.04, 74.77, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월15일', '다윗관부지경계', 0.02, 0.036, 45.43, 56.85, 65.87, 74.8, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.022, 0.058, 48.53, 58.16, 65.6, 70.34, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.035, 0.12, 56.16, 68.84, 68.27, 74.81, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월17일', '다윗관부지경계', 0.01, 0.03, 45.4, 55.26, 73.13, 73.5, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.024, 0.072, 47.82, 62.66, 65.56, 70.93, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.034, 0.162, 58.54, 72.7, 71.29, 74.95, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월20일', '다윗관부지경계', 0.022, 0.03, 42.84, 54.99, 70.51, 74.61, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.031, 0.076, 51.21, 61.1, 64.42, 74.27, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.035, 0.131, 56.61, 70.68, 72.26, 74.91, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월23일', '다윗관부지경계', 0.021, 0.037, 44.7, 58.0, 66.02, 71.84, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.02, 0.068, 45.73, 59.6, 66.66, 71.37, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.049, 0.171, 58.86, 72.1, 70.13, 74.87, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월24일', '다윗관부지경계', nan, 0.021, nan, 41.68, nan, 72.63, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.024, 0.048, 43.03, 56.9, 64.58, 74.31, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.029, 0.083, 57.19, 66.13, 71.26, 74.6, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월30일', '다윗관부지경계', 0.021, 0.045, 45.78, 58.88, 66.79, 72.66, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.055, 0.113, 51.07, 66.73, 71.11, 74.23, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.085, 0.191, 63.92, 72.87, 70.78, 74.66, '0.2cm/s75dB(V)75dB(A)', 'O.K']


In [ ]:
['11월3일', '다윗관부지경계', 0.02, 0.038, 42.8, 57.22, 66.41, 74.58, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.022, 0.1, 52.73, 67.25, 61.26, 71.47, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.045, 0.117, 52.14, 68.11, 69.36, 74.69, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월4일', '다윗관부지경계', 0.021, 0.028, 45.32, 53.89, 68.23, 73.13, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.024, 0.103, 51.0, 69.53, 67.85, 74.68, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.047, 0.145, 58.03, 68.71, 71.03, 74.81, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월7일', '다윗관부지경계', 0.021, 0.032, 45.17, 50.48, 71.0, 74.41, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.068, 0.142, 56.44, 66.99, 71.79, 74.99, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.031, 0.146, 41.79, 71.57, 50.48, 74.57, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월8일', '다윗관부지경계', nan, 0.023, nan, 49.52, nan, 69.46, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.048, 0.093, 52.75, 62.44, 65.51, 74.14, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.056, 0.152, 60.33, 69.93, 69.38, 74.7, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월9일', '다윗관부지경계', nan, 0.021, nan, 44.63, nan, 73.1, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.042, 0.129, 55.02, 68.94, 65.12, 74.71, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.05, 0.137, 59.93, 70.09, 70.96, 74.92, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월14일', '다윗관부지경계', 0.021, 0.066, 42.82, 51.98, 66.77, 73.83, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.02, 0.039, 42.15, 57.58, 66.09, 73.61, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.029, 0.127, 47.57, 66.51, 69.04, 74.77, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월15일', '다윗관부지경계', 0.02, 0.036, 45.43, 56.85, 65.87, 74.8, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.022, 0.058, 48.53, 58.16, 65.6, 70.34, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.035, 0.12, 56.16, 68.84, 68.27, 74.81, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월17일', '다윗관부지경계', 0.01, 0.03, 45.4, 55.26, 73.13, 73.5, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.024, 0.072, 47.82, 62.66, 65.56, 70.93, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.034, 0.162, 58.54, 72.7, 71.29, 74.95, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월20일', '다윗관부지경계', 0.022, 0.03, 42.84, 54.99, 70.51, 74.61, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.031, 0.076, 51.21, 61.1, 64.42, 74.27, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.035, 0.131, 56.61, 70.68, 72.26, 74.91, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월23일', '다윗관부지경계', 0.021, 0.037, 44.7, 58.0, 66.02, 71.84, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.02, 0.068, 45.73, 59.6, 66.66, 71.37, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.049, 0.171, 58.86, 72.1, 70.13, 74.87, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월24일', '다윗관부지경계', nan, 0.021, nan, 41.68, nan, 72.63, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.024, 0.048, 43.03, 56.9, 64.58, 74.31, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.029, 0.083, 57.19, 66.13, 71.26, 74.6, '0.2cm/s75dB(V)75dB(A)', 'O.K', '11월30일', '다윗관부지경계', 0.021, 0.045, 45.78, 58.88, 66.79, 72.66, '0.2cm/s75dB(V)75dB(A)', 'O.K', '주택', 0.055, 0.113, 51.07, 66.73, 71.11, 74.23, '0.2cm/s75dB(V)75dB(A)', 'O.K', '식당부지경계', 0.085, 0.191, 63.92, 72.87, 70.78, 74.66, '0.2cm/s75dB(V)75dB(A)', 'O.K']
